In [1]:
%reload_ext watermark
#%reload_ext lab_black

ModuleNotFoundError: No module named 'watermark'

In [ ]:
%watermark -v -p dateutil,numpy,matplotlib,pandas,torch,tqdm,TaPR_pkg,cv2

In [ ]:
import sys

from pathlib import Path
from datetime import timedelta

import dateutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch

from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import trange
from TaPR_pkg import etapr

In [5]:
TRAIN_DATASET = sorted([x for x in Path("data/dacon/comp5/HAI_2.0/training/").glob("*.csv")])
print(TRAIN_DATASET)

TEST_DATASET = sorted([x for x in Path("data/dacon/comp5/HAI_2.0/testing/").glob("*.csv")])
print(TEST_DATASET)

VALIDATION_DATASET = sorted([x for x in Path("data/dacon/comp5/HAI_2.0/validation/").glob("*.csv")])
print(VALIDATION_DATASET)

NameError: name 'Path' is not defined